# Assignment 5: SMILE Experiment
## Computational Methods in Psychology (and Neuroscience)
### Psychology 4500/7559 --- Fall 2020

# Objectives

Upon completion of this assignment, the student will have:

1. Used the list generation code to make experimental blocks.

2. Created a full-fledged experiment for collecting data.


# Assignment

* Write SMILE code in a Jupyter notebook (after making a copy and renaming it to have your userid in the title --- e.g., A05_SMILE_Experiment_mst3k).

## Details

Your assignment is to turn the lists generated by code from the previous assignment into an experiment. As a reminder, regardless of whether you selected option 1 or option 2, this is a recognition memory experiment. This means that participants will study a list of items one at a time, and then, after a short delay, be tested for their memory of those items. In the test phase of each block, participants will see the study items again, along with an equal number of new items, and for each item they must specify whether the item is an old target item (i.e., one that was on the study list) or a new lure item. 

The high level structure of the experiment is as follows:

- Present the participant some instructions explaining the task
- Optionally provide some practice making responses
- Loop over the blocks of study--test lists

Each block of study--test lists will have the following structure:

- Wait for the participant to press a key to start the block
- Loop over the study list presenting the study items, one at a time
- Wait for a delay (we may eventually fill this with some simple math problems)
- Loop over the test list to present the test items, one at a time, waiting for a keyboard response on each item

Each study item trial will:

- Present the item for a specified duration (this should be a configuration variable at the top of your code)
- Wait an inter-stimulus duration plus some amount of jitter (these, too, should be config variables)
- Log the stimulus information, including when it appeared on the screen

Each test item trial will:

- Present the item on the screen (with either a Label or Image state) until the participant makes a keyboard response of either the key you have selected to indicate the item is "old" or the key that indicates the item is "new"
- Log the stimulus information, including when the stimulus appeared on the screen, when the participant made their response, and what response they made

It is possible to write the entire experiment in one big state machine, but it may be easier to break up these different sections into subroutines.

Be sure to refer to the class notebooks to help guide how to do all the steps above. We have some code below to help you get started.

  
* ***When you are done, save this notebook as HTML (`File -> Download as -> HTML`) and upload it to the matching assignment on UVACollab.***  

In [1]:
import random
from csv import DictReader
import copy

# function to make a study/test block from the pools past in
def gen_block(pools, cond, num_items):
    study_list = []             # fill the study list
    for pool in pools:          # loop over pools
        # loop over items to add from that pool
        # this will be num_items/num_types for mixed lists
        for i in range(num_items):
            study_item = pool.pop()
            study_item.update({'novelty': 'target', 
                               'cond': cond})
            study_list.append(study_item)

    # shuffle the study_list
    random.shuffle(study_list)
    
    # copy the study list to be the start of the test list
    test_list = copy.deepcopy(study_list)
    
    # loop over pools
    for pool in pools:
        # loop over items to add from that pool
        # this will be num_items/num_types for mixed lists
        for i in range(num_items):
            test_item = pool.pop()
            test_item.update({'novelty': 'lure', 
                              'cond': cond})
            test_list.append(test_item)
    
    # shuffle the test list
    random.shuffle(test_list)
    
    return {'study': study_list, 'test': test_list}



In [2]:
# config variables
indoor_file = 'indoor.csv'
outdoor_file = 'outdoor.csv'

# number of pools
num_pools = 2

# number of items in pure lists (must be evenly divisible by num_pools)
num_items_pure = 10

# number of repetitions of each block type
num_reps = 3       

# verify these numbers make sense
num_items_mixed = int(num_items_pure / num_pools)
assert num_items_mixed * num_pools == num_items_pure


# load in the pools
indoor_pool = [i for i in DictReader(open(indoor_file, 'r'))]
outdoor_pool = [i for i in DictReader(open(outdoor_file, 'r'))]


# shuffle the pools
random.shuffle(indoor_pool)
random.shuffle(outdoor_pool)


# generate the blocks
blocks = []
for r in range(num_reps):
    # generate a pure indoor block
    blocks.append(gen_block([indoor_pool], 'indoor', 
                            num_items_pure))
    
    # generate a pure outdoor block
    blocks.append(gen_block([outdoor_pool], 'outdoor', 
                            num_items_pure))
    
    # generate a mixed indoor/outdoor block
    blocks.append(gen_block([indoor_pool, outdoor_pool], 'mixed', 
                            num_items_mixed))

# shuffle the blocks
random.shuffle(blocks)



In [3]:
# **The issue of improper image loading does not seem to appear
# if the cells are all restarted and then run**
# Load in the most common SMILE states
from smile.common import *
from smile.scale import scale as s
from smile.startup import InputSubject


# Configuration for font sizes, key inputs, durations,
# and list of instructions for Scene Study Task
font_size = 50
resp_keys = ['T', 'L']
resp_map = {'target': 'T', 'lure': 'L'}
ISI_dur = 0.5
ISI_jitter = 0.25
inst_font_size = 38
inst_text = """[b][u][size=45]SCENE STUDY INSTRUCTIONS[/size][/b][/u]
In this task, you will study a series of images
Afterwards, you must identify whether 
an image shown in a test was studied or not.
Press T if the image was studied
Press L if the image is new
    
Press SPACEBAR to continue."""


# Create the experiment
exp = Experiment(name='SceneStudy', show_splash=False, 
                 fullscreen=True)


# Create subroutine to display instructions
@Subroutine
def Instruct(self):
    # show the instructions
    Label(text=inst_text, font_size=inst_font_size,
          text_size=(exp.screen.width*0.75, None),
          markup=True, halign='center')
    with UntilDone():
        KeyPress(keys=['SPACEBAR'])


# Create subroutine for study sets to display images for set time,
# wait an interstimulus interval, and log the stimulus information
@Subroutine
def Study(self, block_num, trial_num, cur_trial):
#     Debug(trial_type=cur_trial['in_out'],
#             image_path=cur_trial['in_out'] + '/' + cur_trial['filename'])
    stim = Image(source=(cur_trial['in_out'] + "/" + cur_trial['filename']), 
                 width=400, height=400, allow_stretch=True)
    with UntilDone():
        Wait(ISI_dur*3)
    Wait(ISI_dur, jitter=ISI_jitter)
    Log(name='scene_study',                            # log the result of the trial
        log_dict=cur_trial,
        block_num=block_num,
        trial_num=trial_num,
        stim_on=stim.appear_time
       )
    

# Create subroutine for test sets to display either old target images
# or new lures and wait for one of the configured keypresses to be entered.
# Logs the information included response and response time after. 
@Subroutine
def Test(self, block_num, trial_num, cur_trial):
#     Debug(trial_type=cur_trial['in_out'],
#             image_path=cur_trial['in_out'] + '/' + cur_trial['filename'])
    stim = Image(source=(cur_trial['in_out'] + "/" + cur_trial['filename']), 
                 width=400, height=400, allow_stretch=True)
    with UntilDone():
        Wait(until=stim.appear_time)                  # make sure the stimulus has appeared on the screen
        kp = KeyPress(keys=resp_keys,                 # collect a response (with no timeout)
                      base_time=stim.appear_time['time'],
                      correct_resp=Ref.object(resp_map)[cur_trial['novelty']])
    Wait(ISI_dur, jitter=ISI_jitter)                  # wait the ISI 
    Log(name='scene_test',                            # log the result of the trial
        log_dict=cur_trial,
        block_num=block_num,
        trial_num=trial_num,
        stim_on=stim.appear_time,
        resp=kp.pressed,
        resp_time=kp.press_time,
        rt=kp.rt,
        correct=kp.correct,
       )
    

# Have user enter ID info and then display instructions
InputSubject('Scene Study')
Instruct()                                            
Wait(0.5)


# Establishes loop to go through Study/Test pairs for each condition
# using the set number of reps. Introduces the start of each block
with Loop(blocks) as block:                           
    Label(text='Press the SPACEBAR to\nstart the next block', 
          font_size=font_size, halign='center')
    with UntilDone():
        KeyPress(keys=['SPACEBAR'])
    Wait(ISI_dur, jitter=ISI_jitter)                  # add in some delay before the start of the block
    with Loop(block.current['study']) as study_trial:
        Study(block.i, study_trial.i, study_trial.current)
    Label(text='The test is about to start\n\nRemember\nPress T if the image was studied\nPress L if the image is new\n',
          font_size=font_size, halign='center')
    with UntilDone():
        Wait(ISI_dur*4)                               # wait the ISI 
    with Loop(block.current['test']) as test_trial:
        Test(block.i, test_trial.i, test_trial.current)


# Display termination message at the end of the experiment
Label(text='Congratulations!\nYou have finished the experiment\n\nPress SPACEBAR to exit', 
      font_size=font_size, halign='center')
with UntilDone():
    KeyPress(keys=['SPACEBAR'])

        
# Run the experiment
exp.run()



[INFO   ] [Logger      ] Record log in C:\Users\Carlos Rodriguez\.kivy\logs\kivy_20-11-28_1.txt
[INFO   ] [Kivy        ] v1.11.1
[INFO   ] [Kivy        ] Installed at "C:\ProgramData\Anaconda3\lib\site-packages\kivy\__init__.py"
[INFO   ] [Python      ] v3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]
[INFO   ] [Python      ] Interpreter at "C:\ProgramData\Anaconda3\python.exe"
[INFO   ] [Factory     ] 184 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil, img_gif (img_ffpyplayer ignored)
[INFO   ] [Text        ] Provider: sdl2
[CRITICAL] [Camera      ] Unable to find any valuable Camera provider. Please enable debug logging (e.g. add -d if running from the command line, or change the log level in the config) and re-run your app to identify potential causes
picamera - ModuleNotFoundError: No module named 'picamera'
  File "C:\ProgramData\Anaconda3\lib\site-packages\kivy\core\__init__.py", line 59, in core_select_lib
    mod = __im

In [21]:
import pandas as pd


from smile.log import log2dl
dl=log2dl('data/SceneStudy/s001/20201020_004358/')
dl


df = pd.DataFrame(dl)
df.head(40)



""
